# CHAPTER 14 트리 순회

순회 : 트리 또는 그래프 같은 연결된 구조에서 노드를 방문하는 데 사용되는 알고리즘

### 14.1 깊이 우선 탐색 (DFS)

DFS : 깊이를 우선하여 탐색하는 알고리즘

스택을 사용하여 구현됨

In [ ]:
# 전위 순회: 루트노드 -> 왼쪽노드 -> 오른쪽노드

def preorder(root):
    if root != 0:
        yield root.value
        preorder(root.left)
        preorder(root.right)

In [ ]:
# 후위 순회: 왼쪽노드 -> 오른쪽노드 -> 루트노드

def postorder(root):
    if root != 0:
        postorder(root.left)
        postorder(root.right)
        yield root.value

In [ ]:
# 중위 순회: 왼쪽노드 -> 루트노드 -> 오른쪽노드

def inorder(root):
    if root != 0:
        inorder(root.left)
        yield root.value
        inorder(root.right)

### 14.2 너비 우선 탐색 (BFS)

BFS : 트리 또는 그래프에서 너비를 우선하여 탐색하는 알고리즘

큐를 사용하여 구현됨

### 14.3 트리 순회 구현하기


In [ ]:
# 순회를 위한 BST 클래스(13장) 사용
class Height(object):
    def __init__(self):
        self.height = 0

class NodeBT(object):
    def __init__(self, value=None, level=1):
        self.value = value
        self.level = level
        self.left = None
        self.right = None

    def __repr__(self):
        return "{}".format(self.value)

    def _add_next_node(self, value, level_here=2):
        new_node = NodeBT(value, level_here)
        if not self.value:
            self.value = new_node
        elif not self.left:
            self.left = new_node
        elif not self.right:
            self.right = new_node
        else:
            # 노드에 왼쪽 오른쪽 자식이 모두 있다면,
            # 왼쪽 자식 노드에 새 노드를 추가한다.
            # 그래서 예제의 트리가 왼쪽으로 치우쳐 있다.
            self.left = self.left._add_next_node(value, level_here+1)
        return self

    def _search_for_node(self, value):
        # 전위 순회(pre-order)로 값을 찾는다.
        if self.value == value:
            return self
        else:
            found = None
            if self.left:
                found = self.left._search_for_node(value)
            if self.right:
                found = found or self.right._search_for_node(value)
            return found

    def _is_leaf(self):
        # 왼쪽, 오른쪽 자식이 모두 없는 노드
        return not self.right and not self.left

    def _get_max_height(self):
        # 노드에서 최대 높이를 얻는다 - O(n)
        heightr, heightl = 0, 0
        if self.right:
            heightr = self.right._get_max_height() + 1
        if self.left:
            heightl = self.left._get_max_height() + 1
        return max(heightr, heightl)

    def _is_balanced(self, height=Height()):
        # 균형 트리인지 확인한다 - O(n) ㅋ
        lh = Height()
        rh = Height()

        if self.value is None:
            return True

        l, r = True, True
        if self.left:
            l = self.left._is_balanced(lh)
        if self.right:
            r = self.right._is_balanced(rh)

        height.height = max(lh.height, rh.height) + 1

        if abs(lh.height - rh.height) <= 1:
            return l and r

        return False

    def _is_bst(self, left=None, right=None):
        # 이진 탐색 트리인지 확인한다 - O(n)
        if self.value:
            if left and self.value < left:  # 단축 계산?? ㅇㅇ
                return False
            if right and self.value > right:
                return False

            l, r = True, True
            if self.left:
                l = self.left._is_bst(left, self.value)
            if self.right:
                r = self.right._is_bst(self.value, right)
            return l and r
        else:
            return True


class BinaryTree(object):
    def __init__(self):
        self.root = None

    def add_node(self, value):
        if not self.root:
            self.root = NodeBT(value)
        else:
            self.root._add_next_node(value)

    def is_leaf(self, value):
        node = self.root._search_for_node(value)
        if node:
            return node._is_leaf()
        else:
            return False

    def get_node_level(self, value):
        node = self.root._search_for_node(value)
        if node:
            return node.level
        else:
            return False

    def is_root(self, value):
        return self.root.value == value

    def get_height(self):
        return self.root._get_max_height()

    def is_balanced(self):
        return self.root._is_balanced()

    def is_bst(self):
        return self.root._is_bst()


class NodeBST(NodeBT):
    def __init__(self, value=None, level=1):
        self.value = value
        self.level = level
        self.left = None
        self.right = None

    def _add_next_node(self, value, level_here=2):
        new_node = NodeBST(value, level_here)
        if value > self.value:
            self.right = self.right and self.right._add_next_node(
                value, level_here + 1) or new_node
        elif value < self.value:
            self.left = self.left and self.left._add_next_node(
                value, level_here + 1) or new_node
        else:
            print("중복 노드를 허용하지 않습니다.")
        return self

    def _search_for_node(self, value):
        if self.value == value:
            return self
        elif self.left and value < self.value:
            return self.left._search_for_node(value)
        elif self.right and value > self.value:
            return self.right._search_for_node(value)
        else:
            return False

class BinarySearchTree(BinaryTree):
    def __init__(self):
        self.root = None

    def add_node(self, value):
        if not self.root:
            self.root = NodeBST(value)
        else:
            self.root._add_next_node(value)

In [ ]:
# 반복문을 이용한 이진탐색트리 순회 구현

from collections import deque

class BSTwithTransversalIterative(BinarySearchTree):
    def inorder(self):
        current = self.root
        nodes, stack = [], []
        while stack or current:
            if current:
                stack.append(current)
                current = current.left
            else:
                current = stack.pop()
                nodes.append(current.value)
                current = current.right
        return nodes

    def preorder(self):
        current = self.root
        nodes, stack = [], []
        while stack or current:
            if current:
                nodes.append(current.value)
                stack.append(current)
                current = current.left
            else:
                current = stack.pop()
                current = current.right
        return nodes

    def preorder2(self):
        nodes = []
        stack = [self.root]
        while stack:
            current = stack.pop()
            if current:
                nodes.append(current.value)
                stack.append(current.right)
                stack.append(current.left)
        return nodes

    def BFT(self):  # 너비우선탐색
        current = self.root
        nodes = []
        queue = deque()
        queue.append(current)
        while queue:
            current = queue.popleft()
            nodes.append(current.value)
            if current.left:
                queue.append(current.left)
            if current.right:
                queue.append(current.right)
        return nodes


if __name__ == "__main__":
    bst = BSTwithTransversalIterative()
    l = [10, 5, 6, 3, 8, 2, 1, 11, 9, 4]
    for i in l:
        bst.add_node(i)

    print("노드 8은 말단 노드입니까? ", bst.is_leaf(8))
    print("노드 8의 레벨은? ", bst.get_node_level(8))
    print("노드 10은 루트 노드입니까? ", bst.is_root(10))
    print("노드 1은 루트 노드입니까? ", bst.is_root(1))
    print("트리의 높이는? ", bst.get_height())
    print("이진 탐색 트리입니까? ", bst.is_bst())
    print("균형 트리입니까? ", bst.is_balanced())

    print("전위 순회: ", bst.preorder())
    print("전위 순회2: ", bst.preorder2())
    print("중위 순회: ", bst.inorder())
    print("너비 우선 탐색: ", bst.BFT())

In [ ]:
# 재귀함수를 이용한 이진탐색트리 순회 구현

from collections import deque

class BSTwithTransversalRecursively(BinarySearchTree):

    def __init__(self):
        self.root = None
        self.nodes_BFS = []
        self.nodes_pre = []
        self.nodes_post = []
        self.nodes_in = []

    def BFT(self):      # 너비우선탐색 사용
        self.root.level = 1
        queue = [self.root]
        current_level = self.root.level

        while len(queue) > 0:
            current_node = queue.pop(0)
            if current_node.level > current_level:
                current_level += 1
            self.nodes_BFS.append(current_node.value)

            if current_node.left:
                current_node.left.level = current_level + 1
                queue.append(current_node.left)

            if current_node.right:
                current_node.right.level = current_level + 1
                queue.append(current_node.right)

        return self.nodes_BFS

    def inorder(self, node=None, level=1):
        if not node and level == 1:
            node = self.root
        if node:
            self.inorder(node.left,  level+1)
            self.nodes_in.append(node.value)
            self.inorder(node.right, level+1)
        return self.nodes_in

    def preorder(self, node=None, level=1):
        if not node and level == 1:
            node = self.root
        if node:
            self.nodes_pre.append(node.value)
            self.preorder(node.left, level+1)
            self.preorder(node.right, level+1)
        return self.nodes_pre

    def postorder(self, node=None, level=1):
        if not node and level == 1:
            node = self.root
        if node:
            self.postorder(node.left, level+1)
            self.postorder(node.right, level+1)
            self.nodes_post.append(node.value)
        return self.nodes_post


if __name__ == "__main__":
    bst = BSTwithTransversalRecursively()
    l = [10, 5, 6, 3, 8, 2, 1, 11, 9, 4]
    for i in l:
        bst.add_node(i)

    print("노드 8은 말단 노드입니까? ", bst.is_leaf(8))
    print("노드 8의 레벨은? ", bst.get_node_level(8))
    print("노드 10은 루트 노드입니까? ", bst.is_root(10))
    print("노드 1은 루트 노드입니까? ", bst.is_root(1))
    print("트리의 높이는? ", bst.get_height())
    print("이진 탐색 트리입니까? ", bst.is_bst())
    print("균형 트리입니까? ", bst.is_balanced())

    print("전위 순회: ", bst.preorder())
    print("후위 순회: ", bst.postorder())
    print("중위 순회: ", bst.inorder())
    print("너비 우선 탐색: ", bst.BFT())

### 14.4 연습문제

In [ ]:
# 두 노드의 최소 공통 조상 찾기

def find_ancestor(path, low_value, high_value):  # path == 순회를 한 리스트
    while path:
        current_value = path[0]
        if current_value < low_value:
            try:
                path = path[2:]
            except:
                return current_value
        elif current_value > high_value:
            try:
                path = path[1:]
            except:
                return current_value
        elif low_value <= current_value <= high_value:  # path[0]이 공통조상
            return current_value


if __name__  == "__main__":
    bst = BSTwithTransversalRecursively()
    l = [10,5,6,3,8,2,1,11,9,4]     
    for i in l:
        bst.add_node(i)
    path = bst.preorder()       # [10, 5, 3, 2, 1, 4, 6, 8, 9, 11]
    print("전위 순회: ", path)

    print("1과 6의 최소 공통 조상 : ", find_ancestor(path, 1, 6))
    print("1과 11의 최소 공통 조상:", find_ancestor(path, 1, 11))
    print("1과 4의 최소 공통 조상 : ", find_ancestor(path, 1, 4))
    print("8과 9의 최소 공통 조상 : ", find_ancestor(path, 8, 9))